In [ ]:
!pip install -q git+https://github.com/gmihaila/ml_things.git
!pip install torch-summary
!pip install torchmetrics
!pip install torcheval
!pip install pandasql
!pip install optuna
!pip install optuna.integration

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.6/931.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 107.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 83.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 37.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.2 MB/s eta 0:00:00


In [ ]:
import sys
import importlib
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from torch.utils.data import DataLoader, Dataset
#from transformers import BertTokenizer, BertModel, get_linear_schedule_with_warmup
#from transformers import BertForSequenceClassification
#sys.path.append('E:/Jupyter/notebooks/Thesis/')
#sys.path.append('E:/VSCode-Projects/Thesis/')
sys.path.append('/content/')
import TFIDF_Diagnosis as TFIDF
import BERT_Diagnosis as BD
importlib.reload(BD)
importlib.reload(TFIDF)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


<module 'TFIDF_Diagnosis' from '/content/TFIDF_Diagnosis.py'>

In [ ]:
class BertAndRFClassifier:
    def __init__(self, rf_model, bert_model, device, tokenizer, max_length, batch_size, vectorizer, selector):
        self.rf_model = rf_model
        self.vectorizer = vectorizer
        self.selector = selector
        self.bert_model = bert_model
        self.device = device
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.batch_size = batch_size

    def set_vectorizer(self, vectorizer):
        self.vectorizer = vectorizer

    def set_selector(self, selector):
        self.selector = selector


    def set_rf_model(self, rf_model):
        self.rf_model = rf_model

    def set_bert_model(self, bert_model):
        self.bert_model = bert_model

    def set_device(self, device):
        self.device = device

    def set_vectorizer(self, vectorizer):
        self.vectorizer = vectorizer

    def set_selector(self, selector):
        self.selector = selector

    def fit(self, X, y):
        # Fit Random Forest model on the TF-IDF vectors
        self.rf_model.fit(X, y)

    def predict_proba(self, rf_probs, bert_probs):
        # Predict probabilities using Random Forest on TF-IDF
        rf_probs = self.rf_model.predict_proba(X_tfidf)

        # Predict probabilities using BERT
        bert_probs = self.bert_model.get_test_probs()

        # Average the probabilities (soft-voting)
        combined_probs = (rf_probs + bert_probs) / 2
        return combined_probs

    def predict(self, rf_probs, bert_probs, pcols):

        import pandas as pd
        # Average the probabilities (soft-voting)
        combined_probs = (rf_probs + bert_probs) / 2

        probs_df = pd.DataFrame(combined_probs, columns=pcols)
        probs_df['pred'] = probs_df.idxmax(axis=1)
        print(probs_df[0:5])
        return probs_df

    def predict2(self, test_data, pcols):

        print('got to new predict2')
        import pandas as pd
        # Do RF evauation and get tfidf probabilities
        test_datac = test_data.groupby(['id','icd10_code', 'label'],as_index=False).agg({'text': TFIDF.combine_notes}).reset_index()
        test_data3 = TFIDF.apply_stemmer(test_datac)
        test_X = test_data3.cleaned
        test_y = test_data3.label
        print('executing evaluate')
        rf_probs, rf_preds = TFIDF.evaluate(self.rf_model, test_X, test_y, vectorizer, selector)

        # Do BERT evauation and get probabilities
        bert_notes = BD.create_subsequences(test_data, 318)[['id', 'text', 'label', 'icd10_code']]
        bert_notes = bert_notes.sort_values(by='id')
        test_texts = bert_notes['text'].tolist()
        test_labels = bert_notes['label'].tolist()
        test_adm_ids = bert_notes.id.tolist()
        test_dataset = BD.TextClassificationDataset(test_texts, test_labels, test_adm_ids, self.tokenizer, self.max_length)
        test_dataloader = DataLoader(test_dataset, self.batch_size)
        test_model = self.bert_model.to(self.device)
        test_acc, test_precision, test_sensitivity, test_f1, test_roc_auc, test_cm, test_loss, bert_probs, actuals, bert_preds \
                            = BD.evaluate(test_model, test_dataloader, self.device)
        print('test acc', test_acc, 'test loss', test_loss )
        print('BERT Confusion Matrix:')
        print(test_cm)

        # Average the probabilities (soft-voting)
        combined_probs = (rf_probs + bert_probs) / 2

        probs_df = pd.DataFrame(combined_probs, columns=pcols)
        probs_df['pred'] = probs_df.idxmax(axis=1)
        probs_df['rf_pred'] = rf_preds
        probs_df['bert_pred'] = bert_preds
        print('New probs_df')
        print(probs_df[0:5])
        return probs_df

In [ ]:
class StackingClassifier:
    def __init__(self, rf_model, vectorizer, selector, bert_model, tokenizer, device, max_length, batch_size):
        self.stack_model = LogisticRegression(max_iter=1000)
        #self.stack_model = SVC(kernel='linear', C=100)
        #self.stack_model = RandomForestClassifier(n_estimators=100)
        self.rf_model = rf_model
        self.vectorizer = vectorizer
        self.selector = selector
        self.bert_model = bert_model
        self.device = device
        self.max_length = max_length
        self.batch_size = batch_size
        self.tokenizer = tokenizer

    def set_stack_model(self, stack_model):
        self.stack_model = stack_model

    def set_rf_model(self, rf_model):
        self.rf_model = rf_model

    def set_bert_model(self, bert_model):
        self.bert_model = bert_model

    def set_device(self, device):
        self.device = device

    def fit(self, bert_pred, tfidf_pred, y):
        # Fit stacked model on prior model predictions
        print('bert_pred', len(bert_pred), 'tfidf_pred', len(tfidf_pred), 'labels', len(y))
        print(bert_pred[0:5])
        print(tfidf_pred[0:5])
        print(y[0:5])
        X_val_meta = np.column_stack((bert_pred, tfidf_pred))
        self.stack_model.fit(X_val_meta, y)
        return

    #def predict(self, new_bert_pred, new_tfidf_pred, new_y):
    def predict(self, new_bert_pred, new_tfidf_pred):

        X_val_meta = np.column_stack((new_bert_pred, new_tfidf_pred))
        new_preds = self.stack_model.predict(X_val_meta)
        return new_preds

    def predict2(self, test_data):

        # Do RF evauation and get tfidf probabilities
        test_datac = test_data.groupby(['id','icd10_code', 'label'],as_index=False).agg({'text': TFIDF.combine_notes}).reset_index()
        test_data3 = TFIDF.apply_stemmer(test_datac)
        test_X = test_data3.cleaned
        test_y = test_data3.label
        print('executing tfidf evaluate in stacked predict')
        rf_probs, rf_preds = TFIDF.evaluate(self.rf_model, test_X, test_y, vectorizer, selector)

        # Do BERT evauation and get probabilities
        bert_notes = BD.create_subsequences(test_data, 318)[['id', 'text', 'label', 'icd10_code']]
        bert_notes = bert_notes.sort_values(by='id')
        test_texts = bert_notes['text'].tolist()
        test_labels = bert_notes['label'].tolist()
        test_adm_ids = bert_notes.id.tolist()
        test_dataloader = BD.setup_dataloader(self.tokenizer, test_texts, test_labels, test_adm_ids, self.max_length, self.batch_size, 'test')
        test_model = self.bert_model.to(self.device)

        print('executing BERT evaluate in stacked predict')
        test_acc, test_precision, test_sensitivity, test_f1, test_roc_auc, test_cm, test_loss, bert_probs, bert_actuals, bert_preds \
                            = BD.evaluate(self.bert_model, test_dataloader, self.device)
        print('test acc', test_acc, 'test loss', test_loss )
        print('BERT Confusion Matrix:\n')
        print(test_cm)

        X_val_meta = np.column_stack((bert_preds, rf_preds))
        new_preds = self.stack_model.predict(X_val_meta)
        print('new_preds', len(new_preds), 'bert_preds', len(bert_preds), 'tfidf_preds', len(rf_preds), 'actuals', len(bert_actuals))
        return (new_preds, bert_actuals)


In [ ]:
#def TFIDF_analysis(run, diagnoses_list,breakdown):
def TFIDF_analysis(train_data, test_data, val1_data, val2_data):
# Script flow
    print('executing combine seuqences for TFIDF')

    train_data = TFIDF.read_setup(train_data)
    test_data = TFIDF.read_setup(test_data)
    val1_data = TFIDF.read_setup(val1_data)
    val2_data = TFIDF.read_setup(val2_data)
    print('train', len(train_data), 'test', len(test_data), 'val1', len(val1_data), 'val2', len(val2_data))
    train_data2 = pd.concat([train_data, val1_data], ignore_index=True)
    print('Test counts')
    print(test_data.groupby('icd10_code')['id'].nunique())
    print(train_data2.columns)
    print(test_data.columns)
    print(test_data.groupby('icd10_code')['id'].nunique())

    test_data2 = test_data.sort_values(by='id')
    print('executing apply_stemmer')
    train_data3 = TFIDF.apply_stemmer(train_data2)
    test_data3 = TFIDF.apply_stemmer(test_data)
    print(train_data3.columns)
    print(test_data3.columns)
    X = train_data3.cleaned
    y = train_data3.label
    print('executing train')
    model, vectorizer, selector = TFIDF.train(X, y)
    test_X = test_data3.cleaned
    test_y = test_data3.label
    print('executing evaluate')
    tfidf_probs, preds = TFIDF.evaluate(model, test_X, test_y, vectorizer, selector)
    return(model, vectorizer, selector, tfidf_probs, test_y)

In [ ]:
# generate Stacked Prediction
# this is the good one

def get_stacked_predictions(stacked, run):

    val2_data, labels4 = BD.get_needed_dataset(run, dtype='val2')
    test_data, labels = BD.get_needed_dataset(run, dtype='test')

    val2_data['label'] = val2_data['label'].astype(int)
    test_data['label'] = test_data['label'].astype(int)
    #prepare building stacked model
    print('val2_data in len', len(val2_data), 'test_data in len', len(test_data))
    max_length = 512
    batch_size = 8

    val2_texts = val2_data['text'].tolist()
    print('val_text len', len(val2_texts))
    val2_labels = val2_data['label'].tolist()
    val2_adm_ids = val2_data['id'].tolist()
    val_dataloader = BD.setup_dataloader(tokenizer, val2_texts, val2_labels, val2_adm_ids, max_length, batch_size, 'val')
    #print('dataloader len', len(val_dataloader))
    bert_acc, bert_precision, bert_sensitivity, bert_f1, bert_roc_auc, bert_cm, bert_loss, bert_probs, bert_actuals, bert_preds = BD.evaluate(bert_model, val_dataloader, device)
    print('Bert test acc:', bert_acc)
    print('Bert Confusuon Matrix:')
    print(bert_cm)

    print('starting tfidf on val2 data')
    print('executing apply_stemmer')
    val2_data = val2_data[['id', 'icd10_code', 'text','label']]
    tfidf_val2_data = val2_data.groupby(['id','icd10_code', 'label'],as_index=False).agg({'text': TFIDF.combine_notes}).reset_index()

    tfidf_val2_data = TFIDF.apply_stemmer(tfidf_val2_data)
    #print(tfidf_val_data.columns)
    tfidf_val_X = tfidf_val2_data.cleaned
    tfidf_val_y = tfidf_val2_data.label

    print('executing tfidf evaluate')
    tfidf_probs, tfidf_preds = TFIDF.evaluate(stacked.rf_model, tfidf_val_X, tfidf_val_y, stacked.vectorizer, stacked.selector)
    #print('bert preds', len(bert_preds), 'tfidf preds', len(tfidf_preds), 'actuals', len(bert_actuals))
    print('fit stacked model on original predictions')
    stacked.fit(bert_preds, tfidf_preds, tfidf_val_y)
    print('Generate original model predictions from test data ')
    # Prepare prediction on test dataset
    # test_texts = test_data['text'].tolist()
    # test_labels = test_data['label'].tolist()
    # test_adm_ids = test_data['id'].tolist()
    # print('executing BERT evaluate on Test Data')
    # test_dataloader = BD.setup_dataloader(tokenizer,test_texts, test_labels, test_adm_ids, max_length, batch_size, 'val')
    # new_bert_acc, new_bert_precision, new_bert_sensitivity, new_bert_f1, new_bert_roc_auc, new_bert_cm, new_bert_loss, new_bert_probs, new_bert_actuals, new_bert_preds = BD.evaluate(bert_model, test_dataloader, device)

    # tfidf_test_data = test_data.groupby(['id','icd10_code', 'label'],as_index=False).agg({'text': TFIDF.combine_notes}).reset_index()
    # print('executing apply_stemmer to Test Data')
    # tfidf_test_data = TFIDF.apply_stemmer(tfidf_test_data)
    # print(tfidf_test_data.columns)
    # test_X = tfidf_test_data.cleaned
    # test_y = tfidf_test_data.label
    # print('executing TFIDF evaluate on Test Data')
    # new_tfidf_probs, new_tfidf_preds = TFIDF.evaluate(tfidf_model, test_X, test_y, stacked.vectorizer, stacked.selector)

    print('executing Stacked Predictions')
    y_stacked_preds, test_labels = stacked.predict2(test_data)
    return(y_stacked_preds, test_labels)

#stacked = StackingClassifier(tfidf_model, vectorizer, selector, bert_model, tokenizer, device, max_length, batch_size)
#y_stacked_preds, test_labels = get_stacked_predictions(stacked, val2_data, test_data)
#accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble Stacked', test_labels, y_stacked_preds)
#print('acc', accuracy, '\n', cm )
#print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)





In [ ]:
#  Main flow fron training both classifiers
#  a) train tfidf model and generate predictions on test
#  b) train bert model and generate predictions on test
#  c) run combined preds ensemble predictions
import numpy as np
BD.set_seed(100)
diagnoses_list1 = ['A41.9', 'I21.4', 'J96.00', 'N17.9']
diagnoses_list2 = ['I21.4', 'I25.10', 'I35.2', 'I50.9']
run1 = 'distinct'
run2 = 'similar'
# Distinct Random CV
# hyp_list = [8, 0.00004, 0.85, 0.98, 0, 512, 12]
# Play RSM
#hyp_list = [8, 0.0000197, 0.85, 0.95, 0, 512, 4]
# Distinct RSM
hyp_list = [8, 0.0000197, 0.85, 0.95, 0, 512, 12]
# Similar Random CV
# hyp_list = [8, 0.00002, 0.9, 0.99, 0, 512, 12]
# Similar RSM
# hyp_list = [8, 0.00001, 0.85, 0.95, 0, 512, 12]

breakdown=[110,15,15, 15]
print('Get Data for both models')
train_data, label_dict, test_data, val1_data, val2_data = BD.get_diagnosis_data(diagnoses_list1, run1, breakdown)
print('Running TFIDF Diagnosis Modeling')
tfidf_model, vectorizer, selector, tfidf_probs, actuals = TFIDF_analysis(train_data, test_data, val1_data, val2_data)
print('Running BERT Diagnosis Modeling')

#bert_model,device,tokenizer = BD.one_epoch_set(run1, diagnoses_list1,hyp_list,breakdown)
bert_model,device,tokenizer = BD.one_epoch_set(hyp_list,train_data, label_dict, test_data, val1_data, val2_data)
bert_probs = bert_model.get_test_probs()
max_length = hyp_list[5]
batch_size = hyp_list[0]
print('****************************************')
print('Creating BERT/TFIDF Diagnosis Classifier')
print('****************************************')
eclassifier = BertAndRFClassifier(tfidf_model, bert_model, device, tokenizer, max_length, batch_size, vectorizer, selector)
print('Calculating BERT/TFIDF Diagnosis Predictions')
assignments = [0,1,2,3]
#combined_preds = eclassifier.predict(tfidf_probs, bert_probs, diagnoses_list2)
#combined_preds = eclassifier.predict(tfidf_probs, bert_probs, assignments)
combined_preds = eclassifier.predict2(test_data, assignments)
print('Generatiog BERT/TFIDF Diagnosis Classifier Performance Measures')
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble', actuals, combined_preds.pred)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)
print('****************************************')
print('Creating Stacked Diagnosis Classifier')
print('****************************************')
stacked = StackingClassifier(tfidf_model, vectorizer, selector, bert_model, tokenizer, device, max_length, batch_size)
y_stacked_preds, test_labels = get_stacked_predictions(stacked, run1)
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble Stacked', test_labels, y_stacked_preds)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)
filter = 'A'
print('****************************************')
print('Predicting Ethniciy Data')
print('****************************************')
test_ethnicity(run1, filter, eclassifier, stacked)

Get Data for both models
['A41.9', 'I21.4', 'J96.00', 'N17.9']
distinct


100%|██████████| 440/440 [00:06<00:00, 67.10it/s]


                    id  text
label icd10_code            
0.0   A41.9       1837  1837
1.0   I21.4        922   922
2.0   J96.00      1690  1690
3.0   N17.9       1276  1276


100%|██████████| 60/60 [00:00<00:00, 72.37it/s]


                   id  text
label icd10_code           
0.0   A41.9       221   221
1.0   I21.4       112   112
2.0   J96.00      223   223
3.0   N17.9       264   264


...

100%|██████████| 100/100 [00:01<00:00, 62.22it/s]

setup dataloader input length 1536
executing BERT evaluate in stacked predict


  0%|          | 0/192 [00:00<?, ?it/s]

Val Perf Routine Confusion Matrix:
Epoch Evaluate Performance

 acc 0.75 prec [0.5862069  0.8        0.76666667 0.87096774] sens [0.80952381 0.72727273 0.79310345 0.69230769] f1 [0.68       0.76190476 0.77966102 0.77142857] roc [0.82881254 0.85240041 0.84725595 0.81336696]
test acc 0.75 test loss tensor(3.0806, device='cuda:0')
BERT Confusion Matrix:

[[17  0  3  1]
 [ 1  8  0  2]
 [ 5  0 23  1]
 [ 6  2  4 27]]
new_preds 100 bert_preds 100 tfidf_preds 100 actuals 100
*******************************************
combined preds table
*******************************************
           0         1         2         3  pred  rf_pred  bert_pred  \
0   0.100313  0.019106  0.760524  0.262912     2      2.0          2   
1   0.843857  0.013134  0.019604  0.141587     0      0.0          0   
2   0.072881  0.009669  0.010755  0.906694     3      3.0          3   
3   0.025369  0.754950  0.018319  0.344219     1      1.0          1   
4   0.080178  0.036001  0.843994  0.039828     2      2.0  

,label
0,2
1,0
2,3
3,1
4,2
...,...
95,2
96,3
97,2
98,2


In [ ]:
#filter = 'A'
filter = 'B'

out_actuals = test_ethnicity(run1, filter, eclassifier, stacked)


******************************************
Running Ethnicity Predicting Diagnosis
******************************************
total ethnic adms 70
ethnic_group
B    70
dtype: int64
icd10_code  ethnic_group
A41.9       B               11
I21.4       B                6
J96.00      B               24
N17.9       B               29
dtype: int64
****************************************
Finished Preparing Ethnic Data
****************************************
Ethnic BERT/TFIDF Diagnosis Predictions
got to new predict2
executing evaluate
0    2
1    0
2    3
3    1
4    2
Name: label, dtype: Int64
tf y_pred [2. 0. 3. 1. 2.]
TFIDF Confusion Matrix:
[[10  0  1  0]
 [ 1  4  0  1]
 [ 1  0 22  1]
 [ 0  0  3 26]]
probs dim 2 probs size 280 probs shape (70, 4) probs len 70
TFIDF Performance Measurements
accuracy 0.8857142857142857 precision [0.83333333 1.         0.84615385 0.92857143] 
 sensitivity [0.90909091 0.66666667 0.91666667 0.89655172] f1 [0.86956522 0.8        0.88       0.9122807 ] 
 ROC [0.

100%|██████████| 70/70 [00:01<00:00, 50.95it/s]


  0%|          | 0/162 [00:00<?, ?it/s]

Val Perf Routine Confusion Matrix:
Epoch Evaluate Performance

 acc 0.7285714285714285 prec [0.42105263 1.         0.75       0.91304348] sens [0.72727273 0.66666667 0.75       0.72413793] f1 [0.53333333 0.8        0.75       0.80769231] roc [0.77041602 0.83333333 0.80978261 0.83767872]
test acc 0.7285714285714285 test loss tensor(3.1273, device='cuda:0')
BERT Confusion Matrix:
[[ 8  0  3  0]
 [ 1  4  0  1]
 [ 5  0 18  1]
 [ 5  0  3 21]]
New probs_df
          0         1         2         3  pred  rf_pred  bert_pred
0  0.100313  0.019106  0.760524  0.262912     2      2.0          2
1  0.843857  0.013134  0.019604  0.141587     0      0.0          0
2  0.072881  0.009669  0.010755  0.906694     3      3.0          3
3  0.025369  0.754950  0.018319  0.344219     1      1.0          1
4  0.080178  0.036001  0.843994  0.039828     2      2.0          2
*******************************************
Ethnic BERT/TFIDF Diagnosis Diagnosis Classifier Results
************************************

100%|██████████| 70/70 [00:01<00:00, 50.96it/s]

setup dataloader input length 1291
executing BERT evaluate in stacked predict


  0%|          | 0/162 [00:00<?, ?it/s]

Val Perf Routine Confusion Matrix:
Epoch Evaluate Performance

 acc 0.7285714285714285 prec [0.42105263 1.         0.75       0.91304348] sens [0.72727273 0.66666667 0.75       0.72413793] f1 [0.53333333 0.8        0.75       0.80769231] roc [0.77041602 0.83333333 0.80978261 0.83767872]
test acc 0.7285714285714285 test loss tensor(3.1273, device='cuda:0')
BERT Confusion Matrix:

[[ 8  0  3  0]
 [ 1  4  0  1]
 [ 5  0 18  1]
 [ 5  0  3 21]]
new_preds 70 bert_preds 70 tfidf_preds 70 actuals 70
*******************************************
combined preds table
*******************************************
           0         1         2         3  pred  rf_pred  bert_pred  \
0   0.100313  0.019106  0.760524  0.262912     2      2.0          2   
1   0.843857  0.013134  0.019604  0.141587     0      0.0          0   
2   0.072881  0.009669  0.010755  0.906694     3      3.0          3   
3   0.025369  0.754950  0.018319  0.344219     1      1.0          1   
4   0.080178  0.036001  0.843994  0

In [ ]:
out_actuals[0:5]

,label
0,1
1,1
2,1
3,1
4,1


In [ ]:
def test_ethnicity(run, filter, eclassifier, stacked):
    print('******************************************')
    print('Running Ethnicity Predicting Diagnosis')
    print('******************************************')
    train_id_df = pd.read_csv('/content/aug_train_ids.csv')
    used_ids = train_id_df[['id']].drop_duplicates().values.flatten().tolist()

    if run == 'distinct':
        file7 = '/content/intsl_train_group_full_no2d.snappy.parquet'
    #    file8 = '/content/intsl_test_group_full_no2c.snappy.parquet'
        file31 = '/content/base_adms.snappy.parquet'
    else:
        file7 = '/content/intsl_train_group_full_no3c.snappy.parquet'
        file8 = '/content/intsl_test_group_full_no3c.snappy.parquet'
        file31 = '/content/base_adms.snappy3.parquet'

    in_tr_df = pd.read_parquet(file7, engine='auto', dtype_backend='numpy_nullable')
    #in_tst_df = pd.read_parquet(file8, engine='auto', dtype_backend='numpy_nullable')
    #used_ids[0:5]
    filtered_df = in_tr_df[~in_tr_df['id'].isin(used_ids)]
    #filtered_df = in_tr_df
    #filter_data = pd.concat([filtered_df, in_tst_df])
    #print('rows:', len(filter_data))
    #filter_data.columns

    # get non-whites
    adms = pd.read_parquet(file31, engine='auto', dtype_backend='numpy_nullable')
    adms = adms[['hadm_id', 'ethnicity', 'ethnic_group', 'primary_icd10_code' ]]
    adms.columns = ['id', 'ethnicity', 'ethnic_group', 'icd10_code']
    full_concat = pd.merge(filtered_df, adms, on=['id'], how = 'inner')
    full_concat = full_concat[['id', 'text', 'label', 'icd10_code_x','ethnic_group']]
    full_concat.columns = ['id', 'text', 'label', 'icd10_code','ethnic_group']
    if filter == 'A':
        out_ethnic_notes = full_concat[~full_concat['ethnic_group'].isin(['W','U'])]
    else:
        out_ethnic_notes = full_concat[full_concat['ethnic_group'].isin(['B'])]

    print('total ethnic adms', len(out_ethnic_notes))
    print(out_ethnic_notes[['id', 'ethnic_group']].groupby('ethnic_group').size())
    print(out_ethnic_notes[['icd10_code', 'ethnic_group']].groupby(['icd10_code','ethnic_group']).size())
    #bert_ethnic_notes = BD.create_subsequences(out_ethnic_notes, 318)[['id', 'text', 'label', 'icd10_code']]

    #bert_ethnic_notes = bert_ethnic_notes.sort_values(by='id')
    out_ethnic_notes = out_ethnic_notes.sort_values(by='id')
    rf_ethnic_notes = out_ethnic_notes[['id', 'text', 'label', 'icd10_code']].reset_index()
    actuals = rf_ethnic_notes.label

    print('****************************************')
    print('Finished Preparing Ethnic Data')
    print('****************************************')

    print('Ethnic BERT/TFIDF Diagnosis Predictions')
    assignments = [0,1,2,3]
    combined_preds = eclassifier.predict2(rf_ethnic_notes, assignments)
    combined_preds['tfidf_actuals'] = actuals
    print('*******************************************')
    print('Ethnic BERT/TFIDF Diagnosis Diagnosis Classifier Results')
    print('*******************************************')
    accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble', actuals, combined_preds.pred)
    print('acc', accuracy, '\n', cm )
    print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)
    print('****************************************')
    print('Ethnic Creating Stacked Diagnosis Classifier')
    print('****************************************')
    y_stacked_preds, test_labels = stacked.predict2(rf_ethnic_notes)
    combined_preds['bert_actuals'] = test_labels
    print('*******************************************')
    print('combined preds table')
    print('*******************************************')
    print(combined_preds[0:20])
    print('*******************************************')
    print('Ethnic Stacked Diagnosis Classifier Results')
    print('*******************************************')
    accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble Stacked', test_labels, y_stacked_preds)
    print('acc', accuracy, '\n', cm )
    print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)


    ethnic_results_df = out_ethnic_notes[['id','label', 'icd10_code', 'ethnic_group']]
    ethnic_results_df['stack_pred'] = y_stacked_preds
    ethnic_results_df['avg_pred'] = combined_preds.pred.values
    ethnic_results_df['rf_pred'] = combined_preds.rf_pred.values
    ethnic_results_df['bert_pred'] = combined_preds.bert_pred.values

    #print('Black Results')
    #black_adms = ethnic_results_df[ethnic_results_df['ethnic_group'] == 'B'][['label', 'stack_pred', 'avg_pred', 'rf_pred', 'bert_pred']]
    #black_adms = ethnic_results_df[['label', 'stack_pred', 'avg_pred', 'rf_pred', 'bert_pred']]
    #black_labels = black_adms.label
    #black_stacked_preds = black_adms.stack_pred
    #black_avg_preds = black_adms.avg_pred
    #black_rf_preds = black_adms.rf_pred
    #black_bert_preds = black_adms.bert_pred

    #print('Black RF Results')
    #accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Black RF', black_labels, black_rf_preds)
    #print('acc', accuracy, '\n', cm )
    #print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)

    #print('Black BERT Results')
    #accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Black BERT', black_labels, black_bert_preds)
    #print('acc', accuracy, '\n', cm )
    #print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)

    #print('Black Stacked Ensemble Results')
    #accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Black Stacked', black_labels, black_stacked_preds)
    #print('acc', accuracy, '\n', cm )
    #print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)

    #print('Black Avg Ensemble Results')
    #accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Black Avg Ensemble', black_labels, black_avg_preds)
    #print('acc', accuracy, '\n', cm )
    #print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)
    return (actuals)



In [ ]:
out_ethnic_notes.columns

NameError: name 'out_ethnic_notes' is not defined

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!cp /content/note_data.csv /content/drive/MyDrive/
!cp /content/val1_data.csv /content/drive/MyDrive/
!cp /content/val2_data.csv /content/drive/MyDrive/
!cp /content/test_data.csv /content/drive/MyDrive/

In [ ]:
#from sklearn.ensemble import RandomForestClassifier
stacked = StackingClassifier(tfidf_model, vectorizer, selector, bert_model, tokenizer, device, max_length, batch_size)
y_stacked_preds, test_labels = get_stacked_predictions(stacked, run2)
print('true', test_labels[0:10], 'true', test_labels[0:10])
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Ensemble Stacked', test_labels, y_stacked_preds)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)


100%|██████████| 60/60 [00:00<00:00, 142.67it/s]


                   id  text
label icd10_code           
0.0   I21.4       204   204
1.0   I25.10       68    68
2.0   I35.2        84    84
3.0   I50.9        48    48


100%|██████████| 60/60 [00:00<00:00, 162.70it/s]

                   id  text
label icd10_code           
0.0   I21.4       155   155
1.0   I25.10       65    65
2.0   I35.2        63    63
3.0   I50.9        63    63
val2_data in len 404 test_data in len 346
val_text len 404
setup dataloader input length 404


  0%|          | 0/51 [00:00<?, ?it/s]

Val Perf Routine Confusion Matrix:
Epoch Evaluate Performance

 acc 0.7333333333333333 prec [0.64705882 0.66666667 0.75       0.90909091] sens [0.73333333 0.53333333 1.         0.66666667] f1 [0.6875     0.59259259 0.85714286 0.76923077] roc [0.8        0.72222222 0.94444444 0.82222222]
Bert test acc: 0.7333333333333333
Bert Confusuon Matrix:
[[11  3  1  0]
 [ 2  8  4  1]
 [ 0  0 15  0]
 [ 4  1  0 10]]
starting tfidf on val2 data
executing apply_stemmer
executing tfidf evaluate
0    1
1    3
2    1
3    2
4    3
Name: label, dtype: int64
tf y_pred [0. 3. 1. 2. 3.]
Confusion Matrix:
[[11  3  1  0]
 [ 5  9  1  0]
 [ 1  3 10  1]
 [ 1  1  0 13]]
probs dim 2 probs size 240 probs shape (60, 4) probs len 60
TFIDF Performance Measurements
accuracy 0.7166666666666667 precision [0.61111111 0.5625     0.83333333 0.92857143] 
 sensitivity [0.73333333 0.6        0.66666667 0.86666667] f1 [0.66666667 0.58064516 0.74074074 0.89655172] 
 ROC [0.84888889 0.85777778 0.94074074 0.94518519]
fit stacked mo

  0%|          | 0/44 [00:00<?, ?it/s]

Val Perf Routine Confusion Matrix:
Epoch Evaluate Performance

 acc 0.7666666666666667 prec [0.70588235 0.63157895 0.86666667 1.        ] sens [0.8        0.8        0.86666667 0.6       ] f1 [0.75       0.70588235 0.86666667 0.75      ] roc [0.84444444 0.82222222 0.91111111 0.8       ]
executing apply_stemmer to Test Data
Index(['index', 'id', 'icd10_code', 'label', 'text', 'cleaned'], dtype='object')
executing TFIDF evaluate on Test Data
0    1
1    3
2    2
3    1
4    2
Name: label, dtype: int64
tf y_pred [1. 3. 2. 1. 2.]
Confusion Matrix:
[[ 8  4  0  3]
 [ 2 13  0  0]
 [ 0  0 14  1]
 [ 3  2  0 10]]
probs dim 2 probs size 240 probs shape (60, 4) probs len 60
TFIDF Performance Measurements
accuracy 0.75 precision [0.61538462 0.68421053 1.         0.71428571] 
 sensitivity [0.53333333 0.86666667 0.93333333 0.66666667] f1 [0.57142857 0.76470588 0.96551724 0.68965517] 
 ROC [0.83407407 0.92444444 0.96592593 0.90222222]
executing Stacked Predictions
true 0    1
1    3
2    2
3    1
4   

In [ ]:
print(len(X_meta), len(actuals))

76 76


In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
#from sklearn.linear_model import accuracy_score
X_meta = np.concatenate([tfidf_probs, bert_probs], axis=1)
#X_train_meta, X_val_meta, y_train_meta, y_val_meta = train_test_split(X_meta, actuals, test_size=0.2)

# Train a logistic regression model as the meta-model
meta_model_lr = LogisticRegression(max_iter=1000)
meta_model_lr.fit(X_meta, actuals)

# Make predictions and evaluate
y_pred_meta = meta_model_lr.predict(X_meta)
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Stacked', actuals, y_pred_meta)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)
#accuracy = accuracy_score(y_val_meta, y_pred_meta)
#print(f"Meta-model (Logistic Regression) Accuracy: {accuracy:.4f}")

get perf data input
perf y_true 29    1
53    3
33    2
17    1
38    2
Name: label, dtype: Int64
perf y_pred [1. 3. 2. 1. 2.]
Stacked Perf Routine Confusion Matrix:
y_true 29    1
53    3
33    2
17    1
38    2
Name: label, dtype: Int64
y_pred [1. 3. 2. 1. 2.]
acc 0.7333333333333333 
 [[ 9  4  0  2]
 [ 2 12  0  1]
 [ 0  0 13  2]
 [ 3  2  0 10]]

 acc 0.7333333333333333 prec [0.64285714 0.66666667 1.         0.66666667] sens [0.6        0.8        0.86666667 0.66666667] f1 [0.62068966 0.72727273 0.92857143 0.66666667] roc [0.74444444 0.83333333 0.93333333 0.77777778]


In [ ]:
from sklearn.svm import SVC

# Train a logistic regression model as the meta-model
meta_model_svm = SVC(probability=True)
meta_model_svm.fit(X_meta, actuals)

# Make predictions and evaluate
y_pred_meta_svm = meta_model_svm.predict(X_meta)
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Stacked', actuals, y_pred_meta_svm)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)


19    I25.10
31    I25.10
35    I25.10
44     I35.2
46     I35.2
Name: icd10_code, dtype: string
['I25.10' 'I25.10' 'I25.10' 'I35.2' 'I35.2']
Multi data
acc 0.8026315789473685 
 [[13  2  1  3]
 [ 1 18  0  0]
 [ 0  1 17  1]
 [ 4  2  0 13]]

 acc 0.8026315789473685 prec [0.72222222 0.7826087  0.94444444 0.76470588] sens [0.68421053 0.94736842 0.89473684 0.68421053] f1 [0.7027027  0.85714286 0.91891892 0.72222222] roc [nan nan nan nan]


In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Train a logistic regression model as the meta-model
meta_model_rf = RandomForestClassifier(n_estimators=100)
meta_model_rf.fit(X_meta, actuals)

# Make predictions and evaluate
y_pred_meta_rf = meta_model_rf.predict(X_meta)
accuracy, precision, sensitivity, f1, roc_auc, cm = BD.get_perf_data('Stacked', actuals, y_pred_meta_rf)
print('acc', accuracy, '\n', cm )
print('\n', 'acc', accuracy, 'prec', precision, 'sens', sensitivity, 'f1', f1, 'roc', roc_auc)


19    I25.10
31    I25.10
35    I25.10
44     I35.2
46     I35.2
Name: icd10_code, dtype: string
['I25.10' 'I25.10' 'I25.10' 'I35.2' 'I35.2']
Multi data
acc 1.0 
 [[19  0  0  0]
 [ 0 19  0  0]
 [ 0  0 19  0]
 [ 0  0  0 19]]

 acc 1.0 prec [1. 1. 1. 1.] sens [1. 1. 1. 1.] f1 [1. 1. 1. 1.] roc [nan nan nan nan]


In [ ]:
coefficients = pd.DataFrame({'Feature': X_train.columns, 'Coefficient': tfidf_model.coef_[0]})
print(coefficients) tfidf_model.summary()


val2 420


,label
id,
100139,1
101128,1
101249,1
105844,1
106832,1
112496,1
114791,1
116342,1
118788,1


In [ ]:
test_data[0:5]

,id,text,label,icd10_code
0,182995,chief complaint chest pain nstemi hpi patient ...,0.0,I21.4
1,182995,pump was placed during the procedure and remov...,0.0,I21.4
2,182995,pulmonary embolism bleeding at the time of sur...,0.0,I21.4
3,182995,today after episode of cp with ekg changes and...,0.0,I21.4
4,182995,chf if chronic and improved with diuresis cont...,0.0,I21.4


In [ ]:
combined_preds[0:5]

,I21.4,I25.10,I35.2,I50.9,pred
0,0.123342,0.271328,0.574861,0.030469,I35.2
1,0.598259,0.257561,0.093423,0.342549,I21.4
2,0.414157,0.566591,0.086779,0.035348,I25.10
3,0.233928,0.338384,0.528326,0.272231,I35.2
4,0.120138,0.072371,0.827274,0.041852,I35.2
